# TOS format parser

This is an attempt to get data validation rules from the NHS metadata (the **Technical Output Specification** or TOS.) This is the [Hospital Episode Statistics Data Dictionary
](https://digital.nhs.uk/data-and-information/data-tools-and-services/data-services/hospital-episode-statistics/hospital-episode-statistics-data-dictionary).

In [1]:
# Set Jupyter options
%load_ext autoreload
%autoreload 2

In [3]:
import os

import pandas as pd
import yaml

from field import Field
from rule import Rule

In [7]:
tos_path = os.getenv('TOS_PATH', DEFAULT_TOS_PATH)
tos_file = pd.ExcelFile(tos_path)

Select the tabs we want to work with.

In [8]:
for sheet_name in {s for s in tos_file.sheet_names if 'CUREd' in s}:
    # TODO
    pass

Load the TOS into memory

In [9]:
tos = pd.read_excel(tos_file, sheet_name='HES APC TOS', index_col='Field', skiprows=1)
tos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 379 entries, A_NUMACP to YEAR
Data columns (total 12 columns):
 #   Column                                                                                                                                 Non-Null Count  Dtype 
---  ------                                                                                                                                 --------------  ----- 
 0   Field name                                                                                                                             379 non-null    object
 1   Format                                                                                                                                 379 non-null    object
 2   HES Legacy Field Status (Y/N)                                                                                                          84 non-null     object
 3   Availability                                                            

Build collection of rules from this metadata

In [47]:
for row in tos.sample(5).reset_index().replace(pd.NA, None).to_dict(orient='records'):
    field = Field(
        name=row['Field'],
        title=row['Field name'],
        format_=row['Format'],
        values=row['Values'],
        description=row['Description'],        
    )

    print(field, field.format, sep='\t')
    print(field.values)
    for rule in field.generate_rules():
        print(rule)
    print('_'*128)

FAE	Number
1 = Finished Admission Episode
0 = All other episodes
description: Finished Admission Episode flag is Number
expr: is.integer(FAE)
name: FAE Number



NotImplementedError: 

In [21]:
rules = list()
for row in tos.sample(50).reset_index().replace(pd.NA, None).to_dict(orient='records'):
    print(row['Field'], row['Format'], row['Values'], sep='\t')
    field = Field(
        name=row['Field'],
        title=row['Field name'],
        format_=row['Format'],
        values=row['Values'],
        description=row['Description'],        
    )

    for rule in field.generate_rules():
        rules.append(dict(rule))

    print('____________________')

OACODE01	String(10)	10an = 2001 Census Output Area
Y = not known


NotImplementedError: 

In [22]:
rules_data = yaml.dump(dict(rules=rules))
print(rules_data)

with open('rules.yaml', 'w') as file:
    file.write(rules_data)

rules:
- description: Census Output Area 2001  is String(10)
  expr: is.character(OACODE01) & nchar(OACODE01) == 10
  name: OACODE01 String(10)

